In [1]:
from nemo.collections.asr.models import ASRModel
import torch
from pathlib import Path
import os
import tarfile 
import glob
import shutil


nemo_model_path = '/external/ksingla/models/hf/stt/stt_hi_conformer_ctc_large.nemo'
model_shelf =  "/external2/artifacts/whissle/model_shelf"
save_directory = Path(model_shelf) / "HI_conformer_ctc_large"

os.makedirs(save_directory)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

nemo_model = ASRModel.restore_from(nemo_model_path, map_location=device)

config = {
    "task": "speech_recognition",
    "sample_rate": 16000,
    "encoder.onnx": "model.onnx",
    "tokenizer.model": "tokenizer/tokenizer.model",
    "onnx.intra_op_num_threads": 1,
    "preprocessor": nemo_model.cfg['preprocessor']
}

nemo_model.export(save_directory/"model.onnx")
nemo_archive = tarfile.open(nemo_model_path) 
nemo_archive.extractall(save_directory/"extract") 



[NeMo I 2024-07-05 06:55:47 mixins:172] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2024-07-05 06:55:47 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: ../../data/hindi_normalized/filtered_hindi_v1_manifest.json
    sample_rate: 16000
    batch_size: 24
    shuffle: true
    num_workers: 16
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 30
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: 8
    
[NeMo W 2024-07-05 06:55:47 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /root/ekstep/nemo_exp/vakyansh-nemo-exp

[NeMo I 2024-07-05 06:55:47 features:289] PADDING: 0
[NeMo I 2024-07-05 06:55:48 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /external/ksingla/models/hf/stt/stt_hi_conformer_ctc_large.nemo.


[NeMo W 2024-07-05 06:55:49 nemo_logging:349] /external/svanga/demo/nemo_env/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1716905971132/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
      return F.conv2d(input, weight, bias, self.stride,
    
[NeMo W 2024-07-05 06:55:49 nemo_logging:349] /external/svanga/demo/nemo_env/lib/python3.10/site-packages/torch/onnx/utils.py:2098: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input audio_signal
      warnings.warn(
    
[NeMo W 2024-07-05 06:55:49 nemo_logging:349] /external/svanga/demo/nemo_env/lib/python3.10/site-packages/torch/onnx/utils.py:2098: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input length
  

[NeMo I 2024-07-05 06:56:04 exportable:131] Successfully exported EncDecCTCModelBPE to /external2/artifacts/whissle/model_shelf/HI_conformer_ctc_large/model.onnx


In [7]:
os.makedirs(save_directory/"tokenizer")
tokenizer_model_path = glob.glob(str(save_directory) + "/extract/*tokenizer.model")[0]
shutil.copy(tokenizer_model_path, save_directory/"tokenizer/tokenizer.model")



PosixPath('/external2/artifacts/whissle/model_shelf/HI_conformer_ctc_large/tokenizer/tokenizer.model')

In [11]:
shutil.rmtree(save_directory/"extract")

In [12]:

# Convert dictionary to plain text format
config_text = "\n".join(f"{key}={value}" for key, value in config.items()) + "\n"

# Write the plain text to magic.txt
magic_file = open(save_directory / "magic.txt",'w')
magic_file.write(config_text)
magic_file.close()


In [16]:
config

{'task': 'speech_recognition',
 'sample_rate': 16000,
 'encoder.onnx': 'model.onnx',
 'tokenizer.model': 'tokenizer/tokenizer.model',
 'onnx.intra_op_num_threads': 1,
 'preprocessor': {'_target_': 'nemo.collections.asr.modules.AudioToMelSpectrogramPreprocessor', 'sample_rate': 16000, 'normalize': 'per_feature', 'window_size': 0.025, 'window_stride': 0.01, 'window': 'hann', 'features': 80, 'n_fft': 512, 'log': True, 'frame_splicing': 1, 'dither': 1e-05, 'pad_to': 0, 'pad_value': 0.0}}

In [19]:
yaml.dumps(config)

AttributeError: module 'yaml' has no attribute 'dumps'

In [28]:
config['preprocessor'] = dict(config['preprocessor'])

In [29]:
import yaml
yaml_string = yaml.dump(config, default_flow_style=False)
# with open(save_directory/"magic.yaml", 'w+') as f:
#     yaml.dump(yaml.load(config), f, default_flow_style=False, allow_unicode=True)

In [30]:
yaml_string

'encoder.onnx: model.onnx\nonnx.intra_op_num_threads: 1\npreprocessor:\n  _target_: nemo.collections.asr.modules.AudioToMelSpectrogramPreprocessor\n  dither: 1.0e-05\n  features: 80\n  frame_splicing: 1\n  log: true\n  n_fft: 512\n  normalize: per_feature\n  pad_to: 0\n  pad_value: 0.0\n  sample_rate: 16000\n  window: hann\n  window_size: 0.025\n  window_stride: 0.01\nsample_rate: 16000\ntask: speech_recognition\ntokenizer.model: tokenizer/tokenizer.model\n'